In [ ]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
import re
import csv
import pandas as pd
from bs4 import BeautifulSoup


#Correção do formato do arquivo .xml gerado pelo iTrace#

In [ ]:
def alterar_nomes_xml(arquivo_xml):

    with open(arquivo_xml, 'r') as file:
        conteudo = file.read()

    padrao_abertura = r"<(\d+)>"
    padrao_fechamento = r"</(\d+)>"


    conteudo_modificado = re.sub(padrao_abertura, r"<prefixo_\1>", conteudo)

    conteudo_modificado = re.sub(padrao_fechamento, r"</prefixo_\1>", conteudo_modificado)

    conteudo_modificado = re.sub(r"<\?xml.*?\?>\s*", "", conteudo_modificado, count=1)

    conteudo_modificado = f'<?xml version="1.0" encoding="UTF-8"?>\n<root>\n{conteudo_modificado}\n</root>'

    with open('itraceFixed.xml', 'w') as file:
        file.write(conteudo_modificado)

alterar_nomes_xml('Issue7.xml')

def limpar_nome(nome):
    caracteres_invalidos = ['\\', '/', ':', '*', '?', '"', '<', '>', '|']
    for caractere in caracteres_invalidos:
        nome = nome.replace(caractere, '_')
    return nome

#Armazenamento dos dados em uma lista de listas#

Nesta célula, armazena-se num dicionário os elementos area, id e timestamp do arquivo xml



In [ ]:

data = [
    ["area", "id", "timestamp","url"]
]

i = 1
j = 1
k = 1
m = 1

with open("itraceFixed.xml", 'r', encoding='utf-8') as f:
    xml = minidom.parse(f)
    area = xml.getElementsByTagName("area")
    timestamp = xml.getElementsByTagName("current_timestamp")
    id = xml.getElementsByTagName("id")
    url = xml.getElementsByTagName("url")

    for tag in area:
        if i >= len(data):
          data.append([])
        data[i].insert(0, tag.firstChild.data)
        i += 1

    for tag in timestamp:
       if j >= len(data):
         data.append([])
       data[j].insert(2, tag.firstChild.data)
       j += 1

    for tag in id:
      if k >= len(data):
          data.append([])
      if tag.firstChild == None:
          data[k].insert(1, "None")
      else:
          data[k].insert(1, tag.firstChild.data)
      k += 1

    for tag in url:
        if m  >= len(data):
          data.append([])
        data[m].insert(3,tag.firstChild.data)
        m += 1

#Armazenamento dos dados em um .csv#

In [ ]:

csv_name = "data.csv"

with open (csv_name, mode='w', newline='') as data_csv:

  escritor_csv = csv.writer(data_csv)

  for linha in data:
      escritor_csv.writerow(linha)

#Uso do pandas para análise do .csv#

In [ ]:
df = pd.read_csv(csv_name)
df.head()

,area,id,timestamp,url
0,IssueLabel-\n good first issue\n,label-6cf058,1699997454268,https://github.com/Jaovitosr/Chatbot/issues
1,"IssueOpened-Sep 13, 2023 on Sep 13, 2023, 10:0...",NaN,1699997454749,https://github.com/Jaovitosr/Chatbot/issues
2,"IssueOpened-Sep 13, 2023 on Sep 13, 2023, 10:0...",NaN,1699997454763,https://github.com/Jaovitosr/Chatbot/issues
3,"IssueOpened-Sep 13, 2023 on Sep 13, 2023, 10:0...",NaN,1699997454827,https://github.com/Jaovitosr/Chatbot/issues
4,"IssueOpened-Sep 13, 2023 on Sep 13, 2023, 10:0...",NaN,1699997454882,https://github.com/Jaovitosr/Chatbot/issues


#Contagem do Fixation#

Alem da contagem, caso o conteúdo do campo area seja alguma linha de código, este deve criar um dicionário que corresponde ao id da iteração. Todas as LOC que pertencerem a esse dicionário devem ser armazenadas nele, e caso não haja um dicionario da id em questão, ele deve ser criado



In [ ]:
dic = {}
i=0
chave = "ExtracaoFixation"

#Abre o arquivo csv no modo leitura
with open('data.csv', mode='r', newline='') as csv_file:
    csv_reader = csv.DictReader(csv_file)

    #Itera por cada linha do arquivo csv, pegando os valores das tuplas
    for line in csv_reader:
      area = line['area']
      timestamp = line['timestamp']
      id = line ['id']
      url = line ['url']

      #Caso o campo "area" da tupla contenha uma string com 'LOC', checa se há chave com nome igual ao id da tupla. Caso não haja, cria como uma lista vazia
      if 'LOC' in area:
        if chave not in dic:
          dic[chave] = []

        #Adiciona os valores area, timestamp e id da tupla à lista chave
        dic[chave].append((area, timestamp, id, url))

for chave, valores in dic.items():
    for i in range(len(valores)):
        valores[i] = (valores[i][0], valores[i][1], valores[i][2], valores[i][3], i)


A nova tentativa consiste em implementar um algoritmo que itera por entre os elementos de cada chave criada num dicionário, onde cada chave é um nome de arquivo observado pelo usuário.
O fluxo do algoritmo consiste em:
  - Iterar por cada chave do dicionário
    - Dentro de cada chave, sempre que encontrar um valor[0] não nulo, deve entrar numa série de condicionais:
      -  Caso seja a primeira iteração naquela chave, uma chave de nome igual deve ser criada num novo dicionário e o valor[0] deve ser armazenado numa variável temporária. O dado contido em valor[1] também deve ser armazenado em outra variável temporária, que será o tempo zero detectado naquela aparição do valor de area. Continua-se a iteração pelo dicionário
      - Caso o dado observado em valor[0] seja igual ao nome armazenado na variável temporária, a iteração continua
      - Caso o dado observado em valor[0] seja diferente do nome armazenado na variável temporária, o dado contido em valor[1] é armazenado numa outra variável, contendo o último tempo em que a variável foi observada. Dessa, é subtraída a primeira variável de tempo observada, obtendo a diferença de tempo entre ambas, o que deve ser armazenado na segunda posição do vetor criado com o dado de valor[0], armazenado num primeiro momento


In [ ]:

dic2 = {}

for chave, valores in dic.items():
    time0 = 0 #tempo inicial
    time1 = 0 # tempo final
    time = 0 # tempo total de fixation
    time_aux = 0 #variavel para armazenamento do timestamp da iteração anterior
    val_area = None
    id_atual = None

    # Obtém a quantidade de valores dentro de cada chave
    #qtd_valores = valores
    #previous_area = None

    for valor in valores:
        if area is not None:

            index = valor[3]
            area = valor[0]
            id = valor[2]
            url = valor[3]

            #Essa condicional checa se a area observada na iteração atual é a mesma observada na iteração anterior.
            #Para isso, checo se tanto a area, quanto o arquivo e url da iteração atual são os mesmos da anterior, evitando
            #ocorrências de areas com valor semelhante entre páginas diferentes

            if valor[0] == val_area and valor[2] == id_atual and valor[3] == url_atual:

              #Armazeno o timestamp atual para subtrair do timestamp inicial no momento em que valor[0] != val_area
              time_aux = int(valor[1])

              if chave not in dic2:
                print (chave)
                dic2[chave] = []

            #Quando a area observada na iteração atual for diferente da observada na iteração anterior, os valores são todos
            #atualizados e os calculos de fixation são feitos, subtraindo time_0 de time_aux, evitando fixations acima do normal

            if valor[0] != val_area or valor[2] == id_atual or valor[3] == url_atual:

              #Subtraio o time_aux da iteração anterior com o time_0 coletado na primeira aparição da area observada referente
              #à iteração anterior
              time = time_aux-time0

              aux = [val_area, time, id, url, time0, time_aux]

              if chave not in dic2:
                dic2[chave] = []

              dic2[chave].append(aux)

              val_area = valor[0]
              time0 = int(valor[1])
              time_aux = int(valor[1])
              id_atual = valor[2]
              url_atual = valor[3]


In [ ]:
import csv
import os

def criar_csv(chave, valores, id, url,timestamp0, timestamp1):
    nome_arquivo = chave.replace("/", "_")
    with open(f"{nome_arquivo}.csv", "w", newline="") as arquivo_csv:
        escritor_csv = csv.writer(arquivo_csv)
        escritor_csv.writerow(["area", "fixation", "id", "url", "time0", "time1"])
        for valor in valores:
            if valor[1] >= 100:
                escritor_csv.writerow([valor[0], valor[1], valor[2], valor[3], valor[4], valor[5]])


for chave, valores in dic2.items():
    criar_csv(chave, valores, id, url, time0, time1)



BACKUP DO CODIGO DE EXTRAÇÃO

In [ ]:

dic2 = {}

for chave, valores in dic.items():
    time0 = 0
    time1 = 0
    time = 0
    val_area = None
    id_atual = None

    # Obtém a quantidade de valores dentro de cada chave
    #qtd_valores = valores
    #previous_area = None

    for valor in valores:
        index = valor[3]
        area = valor[0]
        id = valor[2]
        url = valor[3]


        if area is not None:

            if id != id_atual:

              #variável temporária para valor[0]
              val_area = valor[0]

              id_atual = valor[2]
              url_atual = valor[3]
              time0 = int(valor[1])

              if chave not in dic2:
                print (chave)
                dic2[chave] = []

            if valor[0] != val_area:

              time1 = int(valor[1])
              time = time1-time0

              aux = [val_area, time, id, url, time0, time1]

              if chave not in dic2:
                dic2[chave] = []
              dic2[chave].append(aux)

              val_area = valor[0]
              time0 = int(valor[1])
              id_atual = valor[2]
              url_atual = valor[3]
